### Notebook which collects code for image manipulations from Xenium data
- **Developed by:** Anna Maguza
- **Place:** Wuerzburg Institute for System Immunology
- **Date:** 26th March 2024

##### Import packages

+ Single cell packages

In [1]:
import anndata as ad
import scanpy as sc
import squidpy as sq
import pandas as pd
import scipy as sci
from scipy.io import mmread
import matplotlib.pyplot as plt
import scipy as sci
from scipy.sparse import coo_matrix

+ Image manipulation packages

In [2]:
import numpy as np
import tifffile as tf
from scipy.ndimage import affine_transform
from PIL import Image, ImageFile
import json

+ Load ome.tiff image

In [4]:
ome_tiff_path = '/mnt/LaCIE/annaM/gut_project/raw_data/Xenium_10X_datasets/Gut_samples/Non-diseased_pre-designed_pallet/Xenium_V1_hColon_Non_diseased_Base_FFPE_he_image.ome.tif'
img = tf.imread(ome_tiff_path)

+ Convert ome.tiff to png

+ Save level 1 (lowres)

In [5]:
png_path = '/mnt/LaCIE/annaM/gut_project/raw_data/Xenium_10X_datasets/Gut_samples/Non-diseased_pre-designed_and_add-on_panel/Xenium_V1_hColon_Non_diseased_Base_FFPE_he_image_level1.png'

In [6]:
img = tf.imread(ome_tiff_path, series=0, level=1)  

if img.dtype != np.uint8:
    img = (255 * (img - img.min()) / (img.ptp())).astype(np.uint8)

Image.fromarray(img).save(png_path)

print(f"Converted level 1 of {ome_tiff_path} to {png_path}")

Converted level 1 of /mnt/LaCIE/annaM/gut_project/raw_data/Xenium_10X_datasets/Gut_samples/Non-diseased_pre-designed_pallet/Xenium_V1_hColon_Non_diseased_Base_FFPE_he_image.ome.tif to /mnt/LaCIE/annaM/gut_project/raw_data/Xenium_10X_datasets/Gut_samples/Non-diseased_pre-designed_and_add-on_panel/Xenium_V1_hColon_Non_diseased_Base_FFPE_he_image_level1.png


+ Save level 0 (hires)

In [7]:
png_path = '/mnt/LaCIE/annaM/gut_project/raw_data/Xenium_10X_datasets/Gut_samples/Non-diseased_pre-designed_and_add-on_panel/Xenium_V1_hColon_Non_diseased_Base_FFPE_he_image_level0.png'
img = tf.imread(ome_tiff_path, series=0, level=0)  

if img.dtype != np.uint8:
    img = (255 * (img - img.min()) / (img.ptp())).astype(np.uint8)

Image.fromarray(img).save(png_path)

print(f"Converted level 1 of {ome_tiff_path} to {png_path}")

Converted level 1 of /mnt/LaCIE/annaM/gut_project/raw_data/Xenium_10X_datasets/Gut_samples/Non-diseased_pre-designed_pallet/Xenium_V1_hColon_Non_diseased_Base_FFPE_he_image.ome.tif to /mnt/LaCIE/annaM/gut_project/raw_data/Xenium_10X_datasets/Gut_samples/Non-diseased_pre-designed_and_add-on_panel/Xenium_V1_hColon_Non_diseased_Base_FFPE_he_image_level0.png


+ Load png image

In [ ]:
Image.MAX_IMAGE_PIXELS = None
hires = np.asarray(Image.open(png_path))

+ Rotate png image (level - 1)

In [12]:
Image.MAX_IMAGE_PIXELS = None
image_path = '/mnt/LaCIE/annaM/gut_project/raw_data/Xenium_10X_datasets/Gut_samples/Non-diseased_pre-designed_and_add-on_panel/Xenium_V1_hColon_Non_diseased_Base_FFPE_he_image_level1.png'
image = Image.open(image_path)

# Rotate the image
rotated_image = image.rotate(-90)  # Rotate 90 degrees

# Save the rotated image
rotated_image_path = '/mnt/LaCIE/annaM/gut_project/raw_data/Xenium_10X_datasets/Gut_samples/Non-diseased_pre-designed_and_add-on_panel/Xenium_V1_hColon_Non_diseased_Base_FFPE_he_image_level1_rotated.png'
rotated_image.save(rotated_image_path)

+ Rotate png image (level - 0)

In [13]:
Image.MAX_IMAGE_PIXELS = None
image_path = '/mnt/LaCIE/annaM/gut_project/raw_data/Xenium_10X_datasets/Gut_samples/Non-diseased_pre-designed_and_add-on_panel/Xenium_V1_hColon_Non_diseased_Base_FFPE_he_image_level0.png'
image = Image.open(image_path)

# Rotate the image
rotated_image = image.rotate(-90)  # Rotate 90 degrees

# Save the rotated image
rotated_image_path = '/mnt/LaCIE/annaM/gut_project/raw_data/Xenium_10X_datasets/Gut_samples/Non-diseased_pre-designed_and_add-on_panel/Xenium_V1_hColon_Non_diseased_Base_FFPE_he_image_level0_rotated.png'
rotated_image.save(rotated_image_path)

In [ ]:
original_image_path = png_path
rotated_image_path = '/path_to_the_image/rotated_image.png'

In [ ]:
!magick convert original_image_path -rotate -90 rotated_image_path

+ Reduce png image size

In [ ]:
import io

hires_image = Image.fromarray(hires)

buf = io.BytesIO()
hires_image.save(buf, format="PNG", quality=40) 
buf.seek(0)

# Load the image back from the buffer
compressed_image = Image.open(buf)


In [ ]:
# Specify the path where you want to save the compressed image
compressed_image_path = '/path_to_the_image/compressed_image.png'  

# Save the compressed image to the specified path
compressed_image.save(compressed_image_path)

+ Create scale_factors.json for Xenium data

In [ ]:
scale_factors = {
    'level0': {'pixel_size': 0.2125, 'x_pixels': 35416, 'y_pixels': 25778},
    'level1': {'pixel_size': 0.4250, 'x_pixels': 17708, 'y_pixels': 12889},
    'level2': {'pixel_size': 0.8500, 'x_pixels':  8854, 'y_pixels':  6444},
    'level3': {'pixel_size': 1.7000, 'x_pixels':  4427, 'y_pixels':  3222},
    'level4': {'pixel_size': 3.4000, 'x_pixels':  2213, 'y_pixels':  1611},
    'level5': {'pixel_size': 6.8000, 'x_pixels':  1106, 'y_pixels':   805},
    'level6': {'pixel_size':13.6000, 'x_pixels':   553, 'y_pixels':   402},
    'level7': {'pixel_size':27.2000, 'x_pixels':   276, 'y_pixels':   201}
}

json_str = json.dumps(scale_factors, indent=4)

with open('path/scale_factors.json', 'w') as f:
    f.write(json_str)

#### Manual adding of image to the anndata

+ Load h5ad object

In [ ]:
adata_xenium = sc.read_10x_h5('/path/cell_feature_matrix.h5')

reading /mnt/LaCIE/annaM/gut_project/raw_data/Xenium_10X_datasets/Gut_samples/Non-diseased_pre-designed_and_add-on_panel/outs/cell_feature_matrix.h5
 (0:00:00)


+ Load ome.tiff image

In [ ]:
ome_tiff_path = '/path/image.ome.tif'
img = tf.imread(ome_tiff_path)

+ Load coordinates

In [ ]:
coords = pd.read_csv("/path/cells.csv", header=0)

+ Load nucleus boundaries

In [ ]:
nucleus_boundaries = pd.read_csv("/path/nucleus_boundaries.csv",header=0)

+ Load scale factors

In [ ]:
with open('/path/scale_factors.json', 'r') as file:
    scale_factors = json.load(file)

#### Create anndata object

+ Prepare obs

In [ ]:
adata_xenium.obs.index.name = 'cell_id'

+ Prepare uns

In [ ]:
spatial_key = "spatial"
library_id = "tissue42"

In [ ]:
adata_xenium.uns["spatial"] = dict()
adata_xenium.uns["spatial"][library_id] = dict()

+ Add coordinates to obsm

In [ ]:
adata_xenium.obs = adata_xenium.obs.merge(coords, on='cell_id', how='left')

In [ ]:
adata_xenium.obsm["spatial"] = adata_xenium.obs[["x_centroid", "y_centroid"]].copy().to_numpy()

+ Add image to uns

In [ ]:
adata_xenium.uns[spatial_key] = {library_id: {}}
adata_xenium.uns[spatial_key][library_id]["images"] = {}
adata_xenium.uns[spatial_key][library_id]["images"] = {"hires": img}

+ Add scale factors

In [ ]:
adata_xenium.uns[spatial_key][library_id]["scalefactors"] = scale_factors